In [ ]:
class  RawDataUserFilter:

    def __init__(self,textfileloc,numfile,klmfile,pathtofile):
        self.text = textfileloc
        self.num = numfile
        self.klm = klmfile
        self.path = pathtofile
    def textmaker(self):
        #opens and readies a text file containing all filenames
        filenames = open(self.text, 'r')
        filestring = filenames.read()
        listOfFilenames = filestring.split('\n')
        if self.num == 'all':
            listOfFilenames = listOfFilenames
        elif type(self.num)==int:
            listOfFilenames = listOfFilenames[0:self.num]
        elif self.num == "month1":
            listOfFilenames = getOneMonth(listOfFilenames,self.path).month1()
        else:
            print("error a non command value was inserted for numfile")
            listOfFilenames = {}
        return listOfFilenames
    
    
    def klmconverter(self):
        klmNames = (open(self.klm,'r')).read()
        klmNameList = klmNames.split('\n')
        nameList = []
        for item in klmNameList:
            nameList.append(int(float(item)))
        return nameList

    def getReplyKlmBawBB(self):
        KLMlist = RawDataUserFilter(self.text,self.num,self.klm,self.path).klmconverter()
        listOfFilenames = RawDataUserFilter(self.text,self.num,self.klm,self.path).textmaker()
        RepToBawList = []
        KLMframe = []
        for item in listOfFilenames:
            readable = jsonRead(self.path+item).JSONtoDf()
            for entry in readable.iterrows():
                try:
                    alp = int(entry[1]['in_reply_to_user_id'])
                except:
                    continue
                if alp in KLMlist:
                    KLMframe.append(entry)
        
        framelist =[]
        for item in KLMframe:
            framelist.append(item[1])
        RepToKlmList = pandas.DataFrame(framelist)
        RepToBawList.append(readable[readable['in_reply_to_user_id'] == 18332190])
        return(RepToKlmList,RepToBawList)

    def getReplyKlmBaw(self):
        #initialises the list of KLM tweets and files
        KLMlist = RawDataUserFilter(self.text,self.num,self.klm,self.path).klmconverter()
        listOfFilenames = RawDataUserFilter(self.text,self.num,self.klm,self.path).textmaker()
        #two empty lists for storage purposes
        RepToBawList = []
        KLMframe = []
        #reads through every file one at a time
        for item in listOfFilenames:
            readable = jsonRead(self.path+item).JSONtoDf()
            #Checks if the tweet is sent to one of the accounts of KLM
            for entry in readable.iterrows():
                try:
                    alp = int(entry[1]['in_reply_to_user_id'])
                except:
                    continue
                if alp in KLMlist:
                    KLMframe.append(entry)
        #creates a dataframe of tweets sent to klm
        framelist =[]
        for item in KLMframe:
            framelist.append(item[1])
        RepToKlmList = pandas.DataFrame(framelist)
        #gets a list of dataframes for tweets set to british airways
        RepToBawList.append(readable[readable['in_reply_to_user_id'] == 18332190])
        return(RepToKlmList,RepToBawList)
    
    def getKlmAll(self):
        #makes two empty list to dump all the dataframes into one for KLM one for british airways
        KLMlist = RawDataUserFilter(self.text,self.num,self.klm,self.path).klmconverter()
        listOfFilenames = RawDataUserFilter(self.text,self.num,self.klm,self.path).textmaker()
        frameklm = []
        framebrit = []
        KLMframe = []
        #creates a for-loop which sifts through all the files
        for item in listOfFilenames:
            #reminder to see where the system is currently computing
            print("handling dataset:"+item)

            #takes the filename and inserts it into the Json to Dataframe object to turn it into a dataframe
            readable = jsonRead(self.path+item).JSONtoDf()

            #initialises a counter
            usernum=0

            #checks for each user if the value is a dict, if not it gives a corresponding error message
            for i in list(readable['user']):
                if type(i)!= dict:
                    if type(i)==float:
                        print("nan value changed to empty dict")
                    elif type(i)!=float:
                        print('weird value changed to empty dict')
                    #changes non-dict values to be empty
                    readable['user'][usernum] = {}
                usernum+=1

                #takes the user dicts and turns them into rows of a dataframe
            subframe = pandas.DataFrame(list(readable['user']))
            subframe['ident'] = list(readable['id'])
            #a dataframe containing all tweets whose user id is equal to the id for a certain json
            for entry,index in zip(subframe['id'],subframe['ident']):
                try:
                    if int(entry) in KLMlist:
                        KLMframe.append(index)
                except:
                    print('fault')
                    continue
            BritWaylist = list(subframe[subframe['id']==18332190].index)

            #appends all the dataframes per json to the corresponding empty list
            frameklm.append(readable[readable['id'].isin(KLMframe)])
            framebrit.append(readable[readable.index.isin(BritWaylist)])

            #returns the lists of datasets
        return(frameklm,framebrit) 
    
    def getKlmAllBB(self):
        KLMlist = RawDataUserFilter(self.text,self.num,self.klm,self.path).klmconverter()
        listOfFilenames = RawDataUserFilter(self.text,self.num,self.klm,self.path).textmaker()
        frameklm = []
        framebrit = []
        KLMframe = []
        for item in listOfFilenames:
            readable = jsonRead(self.path+item).JSONtoDf()
            usernum=0
            for i in list(readable['user']):
                if type(i)!= dict:
                    readable['user'][usernum] = {}
                usernum+=1
            subframe = pandas.DataFrame(list(readable['user']))
            subframe['ident'] = list(readable['id'])
            for entry,index in zip(subframe['id'],subframe['ident']):
                try:
                    if int(entry) in KLMlist:
                        KLMframe.append(index)     
                except:
                    continue
            BritWaylist = list(subframe[subframe['id']==18332190].index)
            frameklm.append(readable[readable['id'].isin(KLMframe)])
            framebrit.append(readable[readable.index.isin(BritWaylist)])
        return(frameklm,framebrit) 

In [ ]:
class FrameConcat:
    
    def __init__(self,doubleframe):
        self.frames = doubleframe
    
    def concatinator(self):
        concatframes = []
        for i in range(2):
            Rep= self.frames[i]
            try:
                concatframes.append(pandas.concat(Rep,ignore_index=True))
            except:
                concatframes.append(Rep)
        return concatframes


In [ ]:
class getOneMonth:
    
    def __init__(self,file,path):
        self.file = file
        self.path = path
   
    def month1(self):
        Filelist = []
        for item in self.file:
            if 'lastofjune.json' not in Filelist:
                Filelist.append(getOneMonth(item,self.path).jsonwrites())
            else:
                return Filelist
            
    def jsonwrites(self):
        data = []
        with open(self.path+self.file) as f:
            for line in f:
                jsonline =json.loads(line)
                date = jsonline['created_at']
                if date[6] == 'l' and date[9]=='1':
                    output_file = open(self.path+'lastofjune.json','w')
                    for jsonline in data:
                        json.dump(jsonline,output_file)
                        output_file.write("\n")
                    return 'lastofjune.json'
                elif date[6]=='n' and date[8]=='3':
                    data.append(jsonline)
                else:
                    return self.file

In [ ]:
#necessairy libraries
import pandas
import json

#class object
class jsonRead:
        #sets the file location as the self.name
    def __init__(self , filename):
        self.name = filename
        
        #function which reads the json lines and puts them into a dataframe
    def JSONtoDf(self):
        #list to dump json files into
        data = []
        #initialises linecounter for later usage
        linecount = 0
        # takes the self.name opens it and assigns it to the variable f
        with open(self.name) as f:
            #line is one line of json
            for line in f:
                linecount +=1
                #tries to append the json line to the list, if it fails it prints the json that caused the exception
                try:
                    data.append(json.loads(line))
                except:
                    print("fault in line:"+str(linecount))
        #turns the list into a dataframe and returns it
        son = pandas.DataFrame(data)
        return son


In [ ]:
a =RawDataUserFilter('json/filenames.txt','month1','KLMnumb shortlist.txt', 'json/').getKlmAll()
b = FrameConcat(a).concatinator()
b[0]

In [ ]:
b[1]